In [19]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [20]:
import os
import dotenv
from git import Repo

In [21]:
repo_path = "./test_repo"

In [22]:
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [23]:
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/core/langchain_core/",
    glob="**/*",
    suffixes = [".py"],
    exclude=["**/non-utf-8-enconding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents = loader.load()
len(documents)

488

In [24]:
python_spliter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap = 200
)

texts = python_spliter.split_documents(documents)
len(texts)

1475

In [25]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [26]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k": 8},
)

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1000)

In [30]:
prompt = ChatPromptTemplate.from_messages(

    [
        (
            "system",
            "Você e um revisor de código experiente. Forneça informações detalhadas sobre revisão do código e sugestões de melhorias baseadas no contexto fornecido abaixo: \n\n{context}"
        )
    ]

)

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [31]:
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para o código de RunnableBinding"})
response["answer"]

'1. Na função `_is_runnable_type`, a variável `type_` não é um nome descritivo para a variável. É recomendado alterar para um nome mais informativo, como `input_type` ou `model_type`.\n\n2. Na função `bind`, é recomendado adicionar tipo de retorno para a lista, por exemplo `List[Tuple[str, str, str, str]]`, isso vai dar uma ideia mais clara do que é retornado por essa função.\n\n3. Adicionar comentários adequados em cada função/método explicando o que ela faz e quais são os parâmetros esperados.\n\n4. Fornecer exemplos de uso para cada método para melhor compreensão.\n\n5. Utilizar type hints mais detalhados nas funções, como por exemplo especificar os tipos dos parâmetros, dos retornos e das variáveis.\n\n6. Embora o bloco de código esteja bem estr'